In [1]:
using Knet, Test, Base.Iterators, Printf, LinearAlgebra, Random, CUDA, IterTools, DelimitedFiles, Statistics
Knet.atype() = KnetArray{Float32}
using Images, TestImages, OffsetArrays, Colors
using Plots

In [2]:
include("models/transformer.jl")
include("dataloader/dataloader.jl")

prepare_batch (generic function with 1 method)

In [6]:
# read  files
annotations = readdlm("archive/annotations_train.txt", '\t', String, '\n')[2:end,2:end]

# prepare vocab object
bow = ""
for sent in annotations[:,2]#
    bow *= sent
end
#bow = split(bow)
# bow wil be one giant sentence, corpus.
v = Vocab(bow)

# # Initialize Iterators
# img_iterator = ImgReader("archive/img_dirs_val.txt",load_and_process)
# tgt_iterator = TextReader("archive/ann_caps_val.txt",v)

# # simulate retrieving one batch
# img_state, tgt_state = nothing, nothing
# flag = true
# iter = 1
# batch = get_next_batch(img_iterator, tgt_iterator, img_state, tgt_state)

# batch_imgs, batch_indices, img_state, tgt_state = batch
# batch_imgs, batch_indices, labels = prepare_batch(batch, tgt_iterator)
# println(iter)

Vocab(Dict("foal" => 3735, "barbed" => 2735, "bigs" => 4873, "ont" => 4874, "weating" => 4875, "citytwo" => 3736, "enjoy" => 1119, "backgroundfour" => 4876, "fountainmany" => 4877, "chocolate" => 1473…), ["<eos>", "<unk>", "a", "in", "the", "on", "dog", "is", "and", "with"  …  "polebrown", "paintingkid", "backflips", "universitya", "colourfully", "introduces", "joint", "railingman", "frisbree", "panelling"], 2, 1, split)

In [3]:
pretrained = Knet.load("ckpts2/transformer_mlp_512_epoch100.jld2","transformer")

Transformer(512, ImageEncoderMLP(20, 512, Linear(P(Knet.KnetArrays.KnetMatrix{Float32}(512,1200)), P(Knet.KnetArrays.KnetVector{Float32}(512)))), SentEncoder(Embed(P(Knet.KnetArrays.KnetMatrix{Float32}(512,9391)))), EncoderLayer[EncoderLayer(LayerNorm(P(Knet.KnetArrays.KnetMatrix{Float32}(512,1)), P(Knet.KnetArrays.KnetMatrix{Float32}(512,1))), Linear(P(Knet.KnetArrays.KnetMatrix{Float32}(512,512)), P(Knet.KnetArrays.KnetVector{Float32}(512))), Linear(P(Knet.KnetArrays.KnetMatrix{Float32}(512,512)), P(Knet.KnetArrays.KnetVector{Float32}(512))), Linear(P(Knet.KnetArrays.KnetMatrix{Float32}(512,512)), P(Knet.KnetArrays.KnetVector{Float32}(512))), LayerNorm(P(Knet.KnetArrays.KnetMatrix{Float32}(512,1)), P(Knet.KnetArrays.KnetMatrix{Float32}(512,1))), Linear(P(Knet.KnetArrays.KnetMatrix{Float32}(512,512)), P(Knet.KnetArrays.KnetVector{Float32}(512)))), EncoderLayer(LayerNorm(P(Knet.KnetArrays.KnetMatrix{Float32}(512,1)), P(Knet.KnetArrays.KnetMatrix{Float32}(512,1))), Linear(P(Knet.KnetArr

In [4]:
evalon = "val"

"val"

In [7]:
# ONE VALIDATION EPOCH LOOP
pred_corpus = []
img_state, tgt_state = nothing, nothing
img_iterator = ImgReader("archive/img_dirs_$evalon.txt",load_and_process)
tgt_iterator = TextReader("archive/ann_caps_$evalon.txt",v)
while true #iter < 100 #flag !== nothing
    batch = get_next_batch(img_iterator, tgt_iterator, img_state, tgt_state)
    if batch == nothing
        break
    end
    batch_imgs, batch_indices, img_state, tgt_state = batch
    batch_imgs, batch_indices, labels = prepare_batch(batch, tgt_iterator)
    batch_imgs_nomalized = 2.0 .* batch_imgs .- 1.0

    word_probs = pretrained(batch_imgs_nomalized, batch_indices)
    
    argmaxprobs = Array(mapslices(argmax,Array(word_probs),dims=1)[1,:,:])
    pred_words = tgt_iterator.vocab.i2w[argmaxprobs]
    pred_words = permutedims(pred_words,(2,1)) # each sentence prediction is now a row
    
    push!(pred_corpus, pred_words)   

end

In [13]:
open("bleu_files/pred_corpus.txt", "w") do file
    for batch_corpus in pred_corpus
        for sent_i in 1:size(batch_corpus,1)
            for word in batch_corpus[sent_i,:]
                print(file, word, ' ')
            end
            print(file, '\n')
        end
    end
end

In [14]:
# read  files
annotations = readdlm("archive/annotations_$evalon.txt", '\t', String, '\n')[2:end,2:end]

# prepare vocab object
ref_corpus=[]
for sent in annotations[:,2]#
    push!(ref_corpus, sent)
end

open("bleu_files/ref_corpus.txt", "w") do file2
    
    for sent in ref_corpus
        println(file2, sent)
    end
end

Unhandled Task ERROR: InterruptException:
Stacktrace:
 [1] poptask(W::Base.InvasiveLinkedListSynchronized{Task})
   @ Base ./task.jl:827
 [2] wait()
   @ Base ./task.jl:836
 [3] wait(c::Base.GenericCondition{Base.Threads.SpinLock})
   @ Base ./condition.jl:123
 [4] _trywait(t::Timer)
   @ Base ./asyncevent.jl:118
 [5] wait
   @ ./asyncevent.jl:136 [inlined]
 [6] sleep(sec::Int64)
   @ Base ./asyncevent.jl:221
 [7] pool_cleanup()
   @ CUDA ~/.julia/packages/CUDA/t62lT/src/pool.jl:126
 [8] (::CUDA.var"#190#191")()
   @ CUDA ./threadingconstructs.jl:178
